## Final Project Submission

Please fill out:
* Student name: Lameck Odallo
* Student pace: part time 
* Scheduled project review date/time: 27th July 1200
* Instructor name: Christine
* Blog post URL:


In [1]:
# Importing all the relevant packages (Including statistical ones)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid", palette="Set1")
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
%matplotlib inline

In [ ]:
#importing the dataset 
df = pd.read_csv('data/Aviation_Data.csv', low_memory=False)

First Step was to explore the data to check the structure and completeness of data

Conclusion: The dataset contains over 90,000 records and more than 30 variables. Most of the data is in text format, with a few numeric columns. While some fields are mostly complete, others have significant missing data. There’s enough clean and relevant information to support meaningful analysis — particularly the injury counts, aircraft damage classification, and severity of events. Some data points may need to be dropped, but with 90+K observations this may not be a huge risk. 

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90348 entries, 0 to 90347
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      90348 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50249 non-null  object 
 9   Airport.Name            52790 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87572 non-null  object 
 14  Make                    88826 non-null